In [ ]:
import numpy as np
from plots import *
import scipy.io
import pandas as pd
from pathlib import Path

def plot_mags(path, t_win=1, sf=False):
    wf = np.loadtxt(path)
    m = get_mags(wf, sr=44100, t_win=t_win, dict=True)
    mags = m['mags']
    freq_ax = m['freq_ax']
    plt.plot(freq_ax, np.log10(mags)*10)
    plt.title(str(path).split("\\")[-1])
    if sf:
        plt.savefig(str(path).split("\\")[-1].split(".")[0] + ".png")
    plt.show()
def plot_supp(path, save=False):
    data = np.loadtxt(path)
    freqs = data[:, 0]
    mags = data[:, 1]
    plt.plot(freqs, mags)
    plt.title(str(path).split("\\")[-1])
    if save:
        plt.savefig(str(path).split("\\")[-1].split(".")[0] + ".png")
    plt.show()

def has_supptone(data, cutoff=300):
    # find cutoff freq, check if anything above this has > 20 dB
    freqs = data[:, 0]
    mags = data[:, 1]
    # print(np.where(freqs > cutoff))
    i_cutoff = np.where(freqs > cutoff)[0][0]
    if np.max(mags[i_cutoff:]) > 20:
        return True
    else:
        return False
    
    
# get the main directory in my computer
main_path_str = "C:\\Users\\Owner\OneDrive\\Desktop\\SOAE Data\\"
main_path = Path(main_path_str)
# we'll process each subfolder separately since each is likely to have its own quirks

In [ ]:
"""
Plotting particular files
"""
# path1 = "C:\\Users\\Owner\OneDrive\\Desktop\\SOAE Data\\Pre-2014 Data\\Geckos et al MIT\\"
# path2 = "01.26.05\\"
# path1 = r"C:\Users\Owner\OneDrive\Desktop\SOAE Data\Pre-2014 Data\Human (UofA S&A via Wiggio)"
# path2 = "\\07.02.09"
# path1 = r"C:\Users\Owner\OneDrive\Desktop\SOAE Data\Pre-2014 Data\Lizards CUMC2011"
# path2 = r"\05.16.11"
path1 = r"C:\Users\Owner\OneDrive\Desktop\SOAE Data\York Data"
path2 = r"\12.18.14a"
save = False
for fp in Path(path1+path2).rglob('*'):
    fn = fp.name
    ext = fp.suffix
    if fp.is_file() == False:
        continue
    if "README" in fn or ext not in ('.mat', '.txt'):
        print(f"Skipping {fn}")
        continue
    try:
        plot_supp(fp, save=save)
    except:
        print("Waveform detecting, calculating mags")
        print(fp)
        plot_mags(fp, t_win=1, sf=save)

In [ ]:
"""
Testing has_supptone()
"""
folder = "Pre-2014 Data"
    
# We'll build our dataframe by making a dictionary of lists and appending to them
dataframe = {
    'filepath': [],
    'data': [],  
    'species': [],
    'sr': [],
}

# First navigate to our directory
directory_path = Path(main_path_str + folder)

# track which file we're on
n_files = sum(1 for _ in directory_path.rglob('*') if _.is_file())
n_current=0
# now loop through all files in that collection
for fp in directory_path.rglob('*'):
    # Make sure it's a file
    if fp.is_file() == False:  
        continue
    
    # track which file we're on
    n_current += 1
    print(f"Processing file {n_current}/{n_files}")
    
    # Get various versions of the filepath/filename
    
    # Cut off the beginning of the filepath since it's unnecessary for our dataframe (fps = file path shortened)
    fps = str(fp.relative_to(main_path))
    
    # Also get subfolder (if applicable)
    if len(fps.split("\\")) > 1:
        subfolder = fps.split("\\")[1]
    else:
        subfolder = "NA"
        
    # Get the filename itself (without its containing folders), extension, and uppercase version
    fn = fp.name
    ext = fp.suffix
    fnU = fn.upper()
    
    # Treat different files differently based on keywords in filename
    if "README" in fnU or ext not in ('.txt' '.mat'):
        continue
    
    elif "TUBE" in fnU:
        continue
    
    elif "SUPP" in fnU:
        if ext == '.txt':
            data = np.loadtxt(fp)
        else:
            raise ValueError(f"Supp file from {fps} isn't .txt!")
        try: 
            if data.shape[1] != 2:
                raise RuntimeError(f"Supp file from {fps} isn't two columns!")
        except:
            raise(f"Supp file from {fps} isn't 2D!")
        if has_supptone(data):
            plot_supp(fp)
            print("This one has a suppression tone!")
        else:
            plot_supp(fp)
            print("This one doesn't have a suppression tone!")
    if n_current > 10:
        break
        
        
